In [47]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [48]:
from transformers import BertTokenizer
import torch
import pandas as pd


In [49]:
# Load the duc2004 dataset
df = pd.read_csv('DUC/Duc_first_reference/main_dataset/Duc_dataset_first_ref_summary.csv')

In [50]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [51]:
# Tokenize the articles and summaries
input_ids = []
attention_masks = []
for index, row in df.iterrows():
    article = row["Original Article"]
    summary = row["Original Summary"]
    encoded_dict = tokenizer.encode_plus(
                        article,                      # article to encode
                        summary,                      # summary to encode
                        add_special_tokens = True,     # Add '[CLS]' and '[SEP]'
                        max_length = 512,             # Pad/truncate all articles and summaries to 512 tokens
                        pad_to_max_length = True,
                        return_attention_mask = True,  # Generate attention masks
                        return_tensors = 'pt',         # Return PyTorch tensors
                   )
    # Add the encoded article and summary to the lists
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists to tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Print the first 10 encoded articles and summaries
print('Original article: ', df['Original Article'][0])
print('Token IDs: ', input_ids[0])
print('Attention Mask: ', attention_masks[0])
print('Original summary: ', df['Original Summary'][0])
print('Token IDs: ', input_ids[1])
print('Attention Mask: ', attention_masks[1])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you 

Original article:  
Cambodian leader Hun Sen on Friday rejected opposition parties' demands 
for talks outside the country, accusing them of trying to ``internationalize'' 
the political crisis. Government and opposition parties have asked 
King Norodom Sihanouk to host a summit meeting after a series of post-election 
negotiations between the two opposition groups and Hun Sen's party 
to form a new government failed. Opposition leaders Prince Norodom 
Ranariddh and Sam Rainsy, citing Hun Sen's threats to arrest opposition 
figures after two alleged attempts on his life, said they could not 
negotiate freely in Cambodia and called for talks at Sihanouk's residence 
in Beijing. Hun Sen, however, rejected that. ``I would like to make 
it clear that all meetings related to Cambodian affairs must be conducted 
in the Kingdom of Cambodia,'' Hun Sen told reporters after a Cabinet 
meeting on Friday. ``No-one should internationalize Cambodian affairs. 
It is detrimental to the sovereignty of 

In [52]:
model.save_pretrained('finetuned_models/model')
tokenizer.save_pretrained('finetuned_models/model')

('finetuned_models/model\\tokenizer_config.json',
 'finetuned_models/model\\special_tokens_map.json',
 'finetuned_models/model\\vocab.txt',
 'finetuned_models/model\\added_tokens.json')

In [11]:
from transformers import BertForSequenceClassification

# assuming that your fine-tuned model is stored in the `model` variable
model_path = "finetuned_models/model"
model.save_pretrained(model_path)


In [5]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        article = self.df.iloc[idx]["Original Article"]
        summary = self.df.iloc[idx]["Original Summary"]
        
        encoded_input = self.tokenizer(article, summary, truncation=True, padding=True, max_length=512, return_tensors='pt')
        input_ids = encoded_input['input_ids'][0]
        attention_mask = encoded_input['attention_mask'][0]
        labels = torch.tensor(1) # Modify this based on your label format
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

dataset = CustomDataset(df, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [9]:
import torch.optim as optim

optimizer = optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 5
for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        
        optimizer.zero_grad()
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss
        loss.backward()
        optimizer.step()


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [10]:
model.save_pretrained('finetuned_models/bertsum')
tokenizer.save_pretrained('finetuned_models/bertsum')


('finetuned_models/bertsum\\tokenizer_config.json',
 'finetuned_models/bertsum\\special_tokens_map.json',
 'finetuned_models/bertsum\\vocab.txt',
 'finetuned_models/bertsum\\added_tokens.json')

In [11]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('finetuned_models/bertsum')
tokenizer = BertTokenizer.from_pretrained('finetuned_models/bertsum')


In [24]:
input_text = '''
The field of artificial intelligence (AI) has grown significantly in recent years, with advancements in machine learning, natural language processing, and computer vision. AI has a wide range of applications, from image and speech recognition to autonomous vehicles and medical diagnosis. However, with the rapid development of AI comes concerns about its impact on society. One major issue is the potential for AI to replace human jobs, particularly in industries such as manufacturing and customer service. Another concern is the potential for AI to perpetuate bias and discrimination, as algorithms may be trained on biased data. Additionally, there are ethical concerns around the development of autonomous weapons and the use of AI in surveillance. As AI continues to advance, it is important to consider these potential risks and develop strategies to mitigate them.
'''
tokenized_text = tokenizer.tokenize(input_text)

input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors='pt')


In [25]:
import torch

with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs.logits.softmax(dim=-1)
    scores = predictions[:, 1]
    
    # Replace this code with your own sentence extraction algorithm
    sentence_scores = scores.tolist()
    sentences = input_text.split('.')
    
    # Extract top n sentences based on scores
    top_n = 5
    top_indices = sorted(range(len(sentence_scores)), key=lambda i: sentence_scores[i], reverse=True)[:top_n]
    summary = '. '.join([sentences[i] for i in top_indices])


In [26]:
summary

'\nThe field of artificial intelligence (AI) has grown significantly in recent years, with advancements in machine learning, natural language processing, and computer vision'

In [44]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# Initialize the lemmatizer and stopword list
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

# Original sentences
original_text = 'The field of artificial intelligence (AI) has grown significantly in recent years, with advancements in machine learning, natural language processing, and computer vision. AI has a wide range of applications, from image and speech recognition to autonomous vehicles and medical diagnosis. However, with the rapid development of AI comes concerns about its impact on society. One major issue is the potential for AI to replace human jobs, particularly in industries such as manufacturing and customer service. Another concern is the potential for AI to perpetuate bias and discrimination, as algorithms may be trained on biased data. Additionally, there are ethical concerns around the development of autonomous weapons and the use of AI in surveillance. As AI continues to advance, it is important to consider these potential risks and develop strategies to mitigate them'
original_sentences = original_text.split('.')
# Preprocess the sentences
filtered_sentences = []
for sentence in original_sentences:
    # Tokenize the sentence
    words = nltk.word_tokenize(sentence.lower())
    
    # Remove stopwords and lemmatize the remaining words
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords]
    
    # Convert the filtered words back to a sentence and add it to the list of filtered sentences
    filtered_sentence = ' '.join(filtered_words)
    filtered_sentences.append(filtered_sentence)

In [54]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

def generate_summary(filtered_sentences, original_sentences, model_path, tokenizer_path, num_sentences=2):
    # Load the fine-tuned BERT model and tokenizer
    model = BertForSequenceClassification.from_pretrained(model_path)
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    
    # Tokenize the filtered sentences and convert to token IDs
    input_ids = []
    for sentence in filtered_sentences:
        encoded_sentence = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='pt')
        input_ids.append(encoded_sentence)
    
    # Generate sentence scores using the fine-tuned BERT model
    with torch.no_grad():
        scores = []
        for input_id in input_ids:
            outputs = model(input_id)
            predictions = outputs.logits.softmax(dim=-1)
            score = predictions[:, 1]
            scores.append(score)
        
        # Extract top n sentences based on scores
        sentence_scores = torch.cat(scores).tolist()
        top_indices = sorted(range(len(sentence_scores)), key=lambda i: sentence_scores[i], reverse=True)[:num_sentences]
        summary_sentences = [original_sentences[i] for i in top_indices]
        summary = '. '.join(summary_sentences)
        
    return summary


In [56]:
# Assume you have a list of filtered sentences called filtered_sentences and a list of original sentences called original_sentences
model_path = 'finetuned_models/model'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)
num_sentences = 2

summary = generate_summary(filtered_sentences, original_sentences, model_path, model_path, num_sentences)
print(summary)


 Another concern is the potential for AI to perpetuate bias and discrimination, as algorithms may be trained on biased data.  AI has a wide range of applications, from image and speech recognition to autonomous vehicles and medical diagnosis


In [57]:
import pandas as pd
from summarization_algorithm import *
from preprocessing_algorithms import *
from efficiency_scores import *
df = pd.read_csv("DUC\Duc_first_reference\main_dataset\Duc_dataset_first_ref_summary.csv")

In [58]:
sentences, processed_sentences =process_one_column_df(df['Original Article'],remove_stopwords=True,lemmatization=True)

100%|██████████| 49/49 [03:11<00:00,  3.91s/it]


In [60]:
list_of_summaries = []
for i in tqdm(range(49)):
    list_of_summaries.append(generate_summary(sentences[i],processed_sentences[i],model_path, model_path, 5))

100%|██████████| 49/49 [14:21<00:00, 17.58s/it]


In [62]:
df_sum = pd.DataFrame(list_of_summaries)

In [64]:
scores = rouge_scores_df(df,df_sum)

100%|██████████| 49/49 [01:14<00:00,  1.52s/it]


In [65]:
avg_scores = df_avg_by_column(scores)

In [66]:
avg_scores

rouge1_fmeasure        tensor(0.0741)
rouge1_precision       tensor(0.1221)
rouge1_recall          tensor(0.0549)
rouge2_fmeasure        tensor(0.0072)
rouge2_precision       tensor(0.0113)
rouge2_recall          tensor(0.0054)
rougeL_fmeasure        tensor(0.0521)
rougeL_precision       tensor(0.0877)
rougeL_recall          tensor(0.0383)
rougeLsum_fmeasure     tensor(0.0688)
rougeLsum_precision    tensor(0.1138)
rougeLsum_recall       tensor(0.0509)
dtype: object

In [37]:
def get_longest_length(df, series_name):
    max_length = 0
    for item in df[series_name]:
        if len(item) > max_length:
            max_length = len(item)
    return max_length


In [39]:
longest_length = get_longest_length(df, 'Original Article')
longest_length

92941

In [2]:
from transformers import BertTokenizer
from transformers import BertTokenizer
import torch
import pandas as pd

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Load the duc2004 dataset
df = pd.read_csv('DUC/Duc_first_reference/main_dataset/Duc_dataset_first_ref_summary.csv')

In [41]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Load pre-trained tokenizer and model
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')


# Define a custom dataset
class MyDataset(Dataset):
    def __init__(self, df):
        self.articles = df['Original Article'].tolist()
        self.summaries = df['Original Summary'].tolist()
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.articles)
    
    def __getitem__(self, idx):
        article = self.articles[idx]
        summary = self.summaries[idx]
        
        # Tokenize input and output sequences
        input_tokens = self.tokenizer(article, truncation=True, padding='max_length', max_length=1024, return_tensors='pt')
        output_tokens = self.tokenizer(summary, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
        
        # Return input_ids, attention_mask, and decoder_input_ids as well as decoder_attention_mask and labels
        return {'input_ids': input_tokens['input_ids'].squeeze(0), 
                'attention_mask': input_tokens['attention_mask'].squeeze(0),
                'decoder_input_ids': output_tokens['input_ids'].squeeze(0), 
                'decoder_attention_mask': output_tokens['attention_mask'].squeeze(0),
                'labels': output_tokens['input_ids'].squeeze(0)}


# Define train and validation datasets
train_dataset = MyDataset(df[:38])
val_dataset = MyDataset(df[38:])

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

# Define optimizer and learning rate scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

# Train model
for epoch in range(10):
    model.train()
    train_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        decoder_input_ids = batch['decoder_input_ids']
        decoder_attention_mask = batch['decoder_attention_mask']
        labels = batch['labels']
        
        outputs = model(input_ids=input_ids, 
                        attention_mask=attention_mask,
                        decoder_input_ids=decoder_input_ids, 
                        decoder_attention_mask=decoder_attention_mask,
                        labels=labels)
        
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    # Evaluate model on validation dataset
    model.eval()
    val_loss = 0
    for batch in val_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        decoder_input_ids = batch['decoder_input_ids']
        decoder_attention_mask = batch['decoder_attention_mask']
        labels = batch['labels']
        
        with torch.no_grad():
            outputs = model(input_ids=input_ids, 
                            attention_mask=attention_mask,
                            decoder_input_ids=decoder_input_ids, 
                            decoder_attention_mask=decoder_attention_mask,
                            labels=labels)
        
        loss = outputs.loss
        val_loss += loss.item()
    
    # Adjust learning rate
    lr_scheduler.step(val_loss)
    
    # Print train and validation loss for the epoch
    print(f'Epoch {epoch+1}:\nTrain loss: {train_loss/len(train_loader):.4f}\nValidation loss: {val_loss/len(val_loader):.4f}\n')

       


loading file vocab.json from cache at C:\Users\basel/.cache\huggingface\hub\models--facebook--bart-base\snapshots\aadd2ab0ae0c8268c7c9693540e9904811f36177\vocab.json
loading file merges.txt from cache at C:\Users\basel/.cache\huggingface\hub\models--facebook--bart-base\snapshots\aadd2ab0ae0c8268c7c9693540e9904811f36177\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at C:\Users\basel/.cache\huggingface\hub\models--facebook--bart-base\snapshots\aadd2ab0ae0c8268c7c9693540e9904811f36177\config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropo

IndexError: index out of range in self

In [30]:
df['Original Article'][0]

"\r\nCambodian leader Hun Sen on Friday rejected opposition parties' demands \r\nfor talks outside the country, accusing them of trying to ``internationalize'' \r\nthe political crisis. Government and opposition parties have asked \r\nKing Norodom Sihanouk to host a summit meeting after a series of post-election \r\nnegotiations between the two opposition groups and Hun Sen's party \r\nto form a new government failed. Opposition leaders Prince Norodom \r\nRanariddh and Sam Rainsy, citing Hun Sen's threats to arrest opposition \r\nfigures after two alleged attempts on his life, said they could not \r\nnegotiate freely in Cambodia and called for talks at Sihanouk's residence \r\nin Beijing. Hun Sen, however, rejected that. ``I would like to make \r\nit clear that all meetings related to Cambodian affairs must be conducted \r\nin the Kingdom of Cambodia,'' Hun Sen told reporters after a Cabinet \r\nmeeting on Friday. ``No-one should internationalize Cambodian affairs. \r\nIt is detrimenta

In [35]:
text = df['Original Article'][0]
max_length = 512 # or another value lower than 1024
inputs = tokenizer.encode(text, return_tensors='pt')
input_segments = torch.split(inputs, max_length, dim=1)
summary_segments = []
for segment in tqdm(input_segments):
    summary_ids = model.generate(segment,
                                  max_length=max_length,
                                  num_beams=4,
                                  length_penalty=2.0,
                                  no_repeat_ngram_size=3,
                                  early_stopping=True)
    summary_segment = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summary_segments.append(summary_segment)
summary = ' '.join(summary_segments)
print(summary)

















100%|██████████| 14/14 [10:50<00:00, 46.47s/it]

After after after a series of series of post-election election election election elections elections elections election elections election election results election results results results result result result results results outcome outcome outcome outcomes outcome outcomes outcomes outcomes outcome outcome consequences outcomes outcomes consequences consequences consequences outcomes outcome consequences consequences outcome outcomes consequences outcomes consequences effects consequences consequences effects effects effectsEffectsEffectsEffects effects effects effect effect effect effects effect effects effects EffectsEffectsEffects Effects Effects EffectsEffects effectsEffects EffectsEffects Effects effectsEffects effects effectEffectsEffects effect effectsEffects effect effectEffects effects Effects effects effects on the the theTheTheThe TheThe The The TheTheThe theThe The theThe the TheThe the the The The the the,TheThe,The The,The, the the and the the.TheThe.The the, theThe,,Th

In [36]:
summary

'After after after a series of series of post-election election election election elections elections elections election elections election election results election results results results result result result results results outcome outcome outcome outcomes outcome outcomes outcomes outcomes outcome outcome consequences outcomes outcomes consequences consequences consequences outcomes outcome consequences consequences outcome outcomes consequences outcomes consequences effects consequences consequences effects effects effectsEffectsEffectsEffects effects effects effect effect effect effects effect effects effects EffectsEffectsEffects Effects Effects EffectsEffects effectsEffects EffectsEffects Effects effectsEffects effects effectEffectsEffects effect effectsEffects effect effectEffects effects Effects effects effects on the the theTheTheThe TheThe The The TheTheThe theThe The theThe the TheThe the the The The the the,TheThe,The The,The, the the and the the.TheThe.The the, theThe,,T

In [44]:
df['Original Article'][0]

"\r\nCambodian leader Hun Sen on Friday rejected opposition parties' demands \r\nfor talks outside the country, accusing them of trying to ``internationalize'' \r\nthe political crisis. Government and opposition parties have asked \r\nKing Norodom Sihanouk to host a summit meeting after a series of post-election \r\nnegotiations between the two opposition groups and Hun Sen's party \r\nto form a new government failed. Opposition leaders Prince Norodom \r\nRanariddh and Sam Rainsy, citing Hun Sen's threats to arrest opposition \r\nfigures after two alleged attempts on his life, said they could not \r\nnegotiate freely in Cambodia and called for talks at Sihanouk's residence \r\nin Beijing. Hun Sen, however, rejected that. ``I would like to make \r\nit clear that all meetings related to Cambodian affairs must be conducted \r\nin the Kingdom of Cambodia,'' Hun Sen told reporters after a Cabinet \r\nmeeting on Friday. ``No-one should internationalize Cambodian affairs. \r\nIt is detrimenta

In [49]:
text = df['Original Article'][0]
text = '''"\r\nCambodian leader Hun Sen on Friday rejected opposition parties' demands \r\nfor talks outside the country,
 accusing them of trying to ``internationalize'' \r\nthe political crisis. Government and opposition parties have asked 
\r\nKing Norodom Sihanouk to host a summit meeting after a series of post-election \r\nnegotiations between the two opposition
 groups and Hun Sen's party \r\nto form a new government failed. Opposition leaders Prince Norodom \r\nRanariddh and Sam Rainsy,
 citing Hun Sen's threats to arrest opposition \r\nfigures after two alleged attempts on his life, said they could not \r\nnegotiate
 freely in Cambodia and called for talks at Sihanouk's residence \r\nin Beijing. Hun Sen, however, rejected that. ``I would like to make
 \r\nit clear that all meetings related to Cambodian affairs must be conducted \r\nin the Kingdom of Cambodia,'' Hun Sen told reporters 
after a Cabinet \r\nmeeting on Friday. ``No-one should internationalize Cambodian affairs. \r\nIt is detrimental to the sovereignty of Cambodia,
'' he said. Hun Sen's \r\nCambodian People's Party won 64 of the 122 parliamentary seats in \r\nJuly's elections, short of the two-thirds majority
 needed to form \r\na government on its own. Ranariddh and Sam Rainsy have charged that \r\nHun Sen's victory in the elections was achieved through
 widespread \r\nfraud. They have demanded a thorough investigation into their election \r\ncomplaints as a precondition for their cooperation in getting
 the \r\nnational assembly moving and a new government formed. Hun Sen said \r\non Friday that the opposition concerns over their safety in the country 
 \r\nwas ``just an excuse for them to stay abroad.'' Both Ranariddh and \r\nSam Rainsy have been outside the country since parliament was ceremonially \r
 \nopened on Sep. 24. Sam Rainsy and a number of opposition figures have \r\nbeen under court investigation for a grenade attack on Hun Sen's Phnom \r\n
 Penh residence on Sep. 7. Hun Sen was not home at the time of the \r\nattack, which was followed by a police crackdown on demonstrators \r\ncontesting 
 Hun Sen's election victory. The Sam Rainsy Party, in a \r\nstatement released Friday, accused Hun Sen of being ``unwilling to \r\nmake any compromise'' 
 on negotiations to break the deadlock. ``A meeting \r\noutside Cambodia, as suggested by the opposition, could place all \r\nparties on more equal footing
 ,'' said the statement. ``But the ruling \r\nparty refuses to negotiate unless it is able to threaten its negotiating \r\npartners with arrest or worse.'' \r\n \r\nKing Norodom Sihanouk has declined requests to chair a summit of Cambodia's \r\ntop political leaders, saying the meeting would not bring any progress \r\nin deadlocked negotiations to form a government. Cambodian leader \r\nHun Sen's ruling party and the two-party opposition had called on \r\nthe monarch to lead top-level talks, but disagreed on its location. 
 \r\n``Papa will not preside over any summit meeting between the three \r\nparties,whether it is held in Phnom Penh or Beijing, because such \r\na meeting 
 will certainly achieve no result,'' Sihanouk wrote in an \r\nOct. 17 letter to his son, Prince Norodom Ranariddh, leader of the \r\nsenior opposition 
 FUNCINPEC party. A copy of the letter was obtained \r\nThursday. In it, the king called on the three parties to make compromises \r\nto end the stalemate:
  ``Papa would like to ask all three parties to \r\ntake responsibility before the nation and the people.'' Hun Sen used \r\nThursday's anniversary of a
   peace agreement ending the country's civil \r\nwar to pressure the opposition to form a coalition government with \r\nhis party.'''
inputs = tokenizer.encode(text, return_tensors='pt')
summary_ids = model.generate(inputs,
                              max_length=1023,
                              num_beams=4,
                              length_penalty=2.0,
                              no_repeat_ngram_size=3,
                              early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)


" embodiments of the country's �sensitivity to the �escription of the nation's acebookCambodian leader Hun Sen on Friday rejected opposition parties' demands escriptioned to negotiate with the two-partyfor talks outside the country, accusing them of trying to ``internationalize'' ���the political crisis. Government and opposition parties have asked ���sincerity to negotiate peacefully with the Cambodian government. The Hun Sen’s ruling party, in a series of post-election SCP-sensitivity- to- thepolitical crisis, said they could not take any negotiations with the country.The government and opposition groups and Hun Sen's party ߢsincerity, the government of the Kingdom of Cambodia failed. Opposition leaders Prince Norodom �Ranariddh and Sam Rainsy, who has been cited Hun Sen as Hun Sen, in his ��escriptionsensitivityto the political crisis and after two alleged attempts attempts on his life, saidthey could not itiallynegotiate to negotiate freely in Cambodia and called for talks at Sihan

In [50]:
model.save_pretrained(r'finetuned_models\bartsum\saved_model')
# from transformers import BartForConditionalGeneration

# model = BartForConditionalGeneration.from_pretrained('saved_model')


Configuration saved in finetuned_models\bartsum\saved_model\config.json
Model weights saved in finetuned_models\bartsum\saved_model\pytorch_model.bin
